# Purpose: To compare the accuracy levels of a model trained with and without the augmented data set
- Steps:
    0. Initialization
    1. Split testing/training data
    2. Build/reuse a LSTM based classification model, output: 1 out of 3 classes for each test sample + use original dataset, train and test the accuracy level.
    3. Use augmented dataset (augmented rare classes + 80% of original data) for training, use 20% of original data for testing.
    4. Compare accuracy levels of 3 and 4. 

# 0. Initialization

In [1]:
########
# Imports
########
import tensorflow as tf
import keras
import os
import pandas as pd
import src.mutils as util

In [2]:
########
# Check for GPU
########
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# 1. Split testing/training data

In [2]:
########
# Get the number of data rows in the sample dataset
########
util.get_num_labels_in_folder(".\ScooterData", "num_labels", "non_aug_labels_1")
util.get_num_labels_in_folder(".\AugData", "num_labels", "aug_labels_1")

In [2]:
########
# Split the data into train/test samples appropriately -----
########
path_norm = ".\ScooterData"    # Folder path for the non-augmented data
path_aug = ".\AugData"         # Folder path for the augmented data

file_names_norm = util.get_filenames(path_norm)
file_names_aug = util.get_filenames(path_aug)

# Save non-augmented data as a dataframe 
df_normal = pd.concat(
    map(pd.read_csv, file_names_norm), ignore_index=True)
# Remove extra index column
df_normal.pop(df_normal.columns[0])


# Save augmented data as a dataframe 
df_aug = pd.concat(
    map(pd.read_csv, [file for file in file_names_aug]), ignore_index=True)
# Remove extra index column
df_aug.pop(df_aug.columns[0])

# X_train_norm, y_train_norm, X_valid_norm, y_valid_norm, X_test_norm, y_test_norm = util.split_data_with_label(df_normal, 0, 0.3)

# X_train_aug, y_train_aug, X_valid_aug, y_valid_aug, X_test_aug, y_test_aug = util.get_train_test_data(df_normal, 0.2, 0, True, df_aug)
    
    
# test_1 = len(y_train_aug) + len(y_test_aug)
# print(test_1)
# test_2 = len(y_train_norm) + len(y_test_norm)
# print(test_2)
# print(test_1 - test_2)
# print(len(y_train_aug))

['Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimum Sway', 'Minimu

160998
158611
2387
117741


# 2. Train the LSTM classification model w/ original dataset

In [3]:
from src.nn.lstm import LSTM
from src.nn.encoder_decoder import Encoder_Decoder
from src.preprocessor import StableFilter, UnstableFilter
from src.mutils import ModelTrain, split_data, get_filenames

DATA = get_filenames("ScooterData")

DATA_AUG = get_filenames("AugData")
DATA_COMB = DATA + DATA_AUG

OPTIONS = {
    "preprocess": StableFilter(stable_label=0, padding=30),
    "batchsize": 40,
    "timestamp": 16,
    "optimizer": "adam",
}

MAX_EPOCHS = 30

number_of_test_file = 4
stable_test_data = StableFilter(stable_label=0, padding=30).transform(split_data(DATA[:number_of_test_file], 0, 0)[0])
unstable_test_data = UnstableFilter(stable_label=0, padding=10).transform(split_data(DATA[:number_of_test_file], 0, 0)[0])

SETTINGS = {
    "max_epochs":MAX_EPOCHS,
    "valid_ratio":0.3,
    "test_ratio":0,
    "early_stop_valid_patience":MAX_EPOCHS//10,
    "early_stop_train_patience":MAX_EPOCHS//10,
    "num_train_per_config":10,
    "loss":'mae',
    "metrics": ['mae'],
    # "loss":"sparse_categorical_crossentropy",
    "metrics": ['accuracy'],
    "verbose": 1,
    "test_data": [unstable_test_data, stable_test_data]
}

ModelTrain(Encoder_Decoder, DATA[number_of_test_file:], OPTIONS, **SETTINGS).run()

batchsize   : 40
timestamp   : 16
optimizer   : adam
preprocess  : Stable Filter

(4957, 16, 27)
Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 16, 27)]          0         
                                                                 
 lstm_2 (LSTM)               (None, 10)                1520      
                                                                 
 repeat_vector (RepeatVector  (None, 16, 10)           0         
 )                                                               
                                                                 
 lstm_3 (LSTM)               (None, 16, 27)            4104      
                                                                 
Total params: 5,624
Trainable params: 5,624
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
124/124 [======

In [4]:
from nn.lstm import LSTM
from mutils import ModelTest, split_data, get_filenames
from preprocessor import Balancer, Jitter, StableFilter, UnstableFilter
from nn.encoder_decoder import Encoder_Decoder



MAX_EPOCHS = 30

OPTIONS = {
    "preprocess": [StableFilter(stable_label=0, padding=30)],
    "batchsize": [40],
    "timestamp": [16],
    "optimizer": ["adam"],
    "layer1": [{"units": i*5} for i in range(1, 10)],
}

number_of_test_file = 4
stable_test_data = StableFilter(stable_label=0, padding=30).transform(split_data(DATA[:number_of_test_file], 0, 0)[0])
unstable_test_data = UnstableFilter(stable_label=0, padding=10).transform(split_data(DATA[:number_of_test_file], 0, 0)[0])

SETTINGS = {
    "max_epochs":MAX_EPOCHS,
    "valid_ratio":0.3,
    "test_ratio":0,
    "early_stop_valid_patience":MAX_EPOCHS//10,
    "early_stop_train_patience":MAX_EPOCHS//10,
    "num_train_per_config":10,
    "loss":'mae',
    "metrics": ['mae'],
    # "loss":"sparse_categorical_crossentropy",
    "metrics": ['accuracy'],
    "verbose": 1,
    "test_data": [unstable_test_data, stable_test_data]
}

ModelTest(Encoder_Decoder, DATA[number_of_test_file:], OPTIONS, **SETTINGS).run()

preprocess  : Stable Filter
batchsize   : 40
timestamp   : 16
optimizer   : adam
layer1      : {'units': 5}

(4957, 16, 27)
Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 16, 27)]          0         
                                                                 
 lstm_6 (LSTM)               (None, 5)                 660       
                                                                 
 repeat_vector_1 (RepeatVect  (None, 16, 5)            0         
 or)                                                             
                                                                 
 lstm_7 (LSTM)               (None, 16, 27)            3564      
                                                                 
Total params: 4,224
Trainable params: 4,224
Non-trainable params: 0
_________________________________________________________________

Epoch 20/30
124/124 [==============================] - 1s 10ms/step - loss: 0.0293 - accuracy: 0.6334 - val_loss: 0.0294 - val_accuracy: 0.5215
Epoch 21/30
124/124 [==============================] - 1s 10ms/step - loss: 0.0289 - accuracy: 0.6374 - val_loss: 0.0288 - val_accuracy: 0.5351
Epoch 22/30
124/124 [==============================] - 1s 10ms/step - loss: 0.0286 - accuracy: 0.6424 - val_loss: 0.0282 - val_accuracy: 0.5684
Epoch 23/30
124/124 [==============================] - 1s 9ms/step - loss: 0.0283 - accuracy: 0.6468 - val_loss: 0.0277 - val_accuracy: 0.6019
Epoch 24/30
124/124 [==============================] - 1s 10ms/step - loss: 0.0280 - accuracy: 0.6537 - val_loss: 0.0272 - val_accuracy: 0.6328
Epoch 25/30
124/124 [==============================] - 1s 11ms/step - loss: 0.0278 - accuracy: 0.6612 - val_loss: 0.0268 - val_accuracy: 0.6639
Epoch 26/30
124/124 [==============================] - 2s 14ms/step - loss: 0.0276 - accuracy: 0.6681 - val_loss: 0.0264 - val_accuracy: 

124/124 [==============================] - 1s 12ms/step - loss: 0.0291 - accuracy: 0.6431 - val_loss: 0.0288 - val_accuracy: 0.4278
Epoch 18/30
124/124 [==============================] - 1s 12ms/step - loss: 0.0286 - accuracy: 0.6442 - val_loss: 0.0280 - val_accuracy: 0.4470
Epoch 19/30
124/124 [==============================] - 1s 12ms/step - loss: 0.0282 - accuracy: 0.6431 - val_loss: 0.0273 - val_accuracy: 0.4714
Epoch 20/30
124/124 [==============================] - 1s 12ms/step - loss: 0.0278 - accuracy: 0.6459 - val_loss: 0.0267 - val_accuracy: 0.5161
Epoch 21/30
124/124 [==============================] - 1s 12ms/step - loss: 0.0274 - accuracy: 0.6508 - val_loss: 0.0261 - val_accuracy: 0.5774
Epoch 22/30
124/124 [==============================] - 1s 12ms/step - loss: 0.0271 - accuracy: 0.6575 - val_loss: 0.0256 - val_accuracy: 0.6433
Epoch 23/30
124/124 [==============================] - 1s 12ms/step - loss: 0.0267 - accuracy: 0.6651 - val_loss: 0.0251 - val_accuracy: 0.6765
Epoc

124/124 [==============================] - 1s 11ms/step - loss: 0.0332 - accuracy: 0.6128 - val_loss: 0.0340 - val_accuracy: 0.4813
Epoch 15/30
124/124 [==============================] - 1s 11ms/step - loss: 0.0325 - accuracy: 0.6154 - val_loss: 0.0334 - val_accuracy: 0.5299
Epoch 16/30
124/124 [==============================] - 1s 11ms/step - loss: 0.0318 - accuracy: 0.6258 - val_loss: 0.0327 - val_accuracy: 0.6170
Epoch 17/30
124/124 [==============================] - 1s 11ms/step - loss: 0.0312 - accuracy: 0.6313 - val_loss: 0.0322 - val_accuracy: 0.6301
Epoch 18/30
124/124 [==============================] - 1s 10ms/step - loss: 0.0307 - accuracy: 0.6345 - val_loss: 0.0317 - val_accuracy: 0.6297
Epoch 19/30
124/124 [==============================] - 1s 10ms/step - loss: 0.0302 - accuracy: 0.6406 - val_loss: 0.0312 - val_accuracy: 0.6323
Epoch 20/30
124/124 [==============================] - 1s 11ms/step - loss: 0.0297 - accuracy: 0.6485 - val_loss: 0.0307 - val_accuracy: 0.6359
Epoc

124/124 [==============================] - 2s 12ms/step - loss: 0.0315 - accuracy: 0.6275 - val_loss: 0.0281 - val_accuracy: 0.6088
Epoch 12/30
124/124 [==============================] - 2s 12ms/step - loss: 0.0307 - accuracy: 0.6287 - val_loss: 0.0274 - val_accuracy: 0.6145
Epoch 13/30
124/124 [==============================] - 2s 12ms/step - loss: 0.0301 - accuracy: 0.6321 - val_loss: 0.0268 - val_accuracy: 0.6243
Epoch 14/30
124/124 [==============================] - 2s 13ms/step - loss: 0.0296 - accuracy: 0.6320 - val_loss: 0.0263 - val_accuracy: 0.6313
Epoch 15/30
124/124 [==============================] - 2s 13ms/step - loss: 0.0291 - accuracy: 0.6344 - val_loss: 0.0258 - val_accuracy: 0.6356
Epoch 16/30
124/124 [==============================] - 1s 12ms/step - loss: 0.0287 - accuracy: 0.6367 - val_loss: 0.0253 - val_accuracy: 0.6364
Epoch 17/30
124/124 [==============================] - 2s 12ms/step - loss: 0.0283 - accuracy: 0.6388 - val_loss: 0.0249 - val_accuracy: 0.6403
Epoc

...

Epoch 28/30
124/124 [==============================] - 1s 10ms/step - loss: 0.0174 - accuracy: 0.8206 - val_loss: 0.0137 - val_accuracy: 0.8133
Epoch 28: early stopping
Epoch 1/30
124/124 [==============================] - 4s 16ms/step - loss: 0.0600 - accuracy: 0.5439 - val_loss: 0.0373 - val_accuracy: 0.3621
Epoch 2/30
124/124 [==============================] - 1s 11ms/step - loss: 0.0397 - accuracy: 0.5712 - val_loss: 0.0321 - val_accuracy: 0.3921
Epoch 3/30
124/124 [==============================] - 1s 10ms/step - loss: 0.0358 - accuracy: 0.5974 - val_loss: 0.0292 - val_accuracy: 0.4334
Epoch 4/30
124/124 [==============================] - 1s 10ms/step - loss: 0.0319 - accuracy: 0.6412 - val_loss: 0.0258 - val_accuracy: 0.6813
Epoch 5/30
124/124 [==============================] - 1s 10ms/step - loss: 0.0282 - accuracy: 0.6523 - val_loss: 0.0244 - val_accuracy: 0.7261
Epoch 6/30
124/124 [==============================] - 1s 11ms/step - loss: 0.0261 - accuracy: 0.6639 - val_loss: 0.0

124/124 [==============================] - 1s 11ms/step - loss: 0.0356 - accuracy: 0.6302 - val_loss: 0.0341 - val_accuracy: 0.3622
Epoch 4/30
124/124 [==============================] - 1s 11ms/step - loss: 0.0324 - accuracy: 0.6160 - val_loss: 0.0319 - val_accuracy: 0.3615
Epoch 5/30
124/124 [==============================] - 1s 11ms/step - loss: 0.0291 - accuracy: 0.6356 - val_loss: 0.0280 - val_accuracy: 0.4404
Epoch 6/30
124/124 [==============================] - 1s 11ms/step - loss: 0.0272 - accuracy: 0.6633 - val_loss: 0.0238 - val_accuracy: 0.6978
Epoch 7/30
124/124 [==============================] - 1s 10ms/step - loss: 0.0253 - accuracy: 0.6775 - val_loss: 0.0214 - val_accuracy: 0.7696
Epoch 8/30
124/124 [==============================] - 1s 10ms/step - loss: 0.0238 - accuracy: 0.6887 - val_loss: 0.0196 - val_accuracy: 0.7597
Epoch 9/30
124/124 [==============================] - 1s 11ms/step - loss: 0.0229 - accuracy: 0.7317 - val_loss: 0.0184 - val_accuracy: 0.7817
Epoch 10/3

Epoch 6/30
124/124 [==============================] - 1s 10ms/step - loss: 0.0269 - accuracy: 0.6420 - val_loss: 0.0228 - val_accuracy: 0.7057
Epoch 7/30
124/124 [==============================] - 1s 10ms/step - loss: 0.0253 - accuracy: 0.6523 - val_loss: 0.0221 - val_accuracy: 0.7381
Epoch 8/30
124/124 [==============================] - 1s 10ms/step - loss: 0.0241 - accuracy: 0.6742 - val_loss: 0.0210 - val_accuracy: 0.7507
Epoch 9/30
124/124 [==============================] - 1s 10ms/step - loss: 0.0232 - accuracy: 0.6914 - val_loss: 0.0202 - val_accuracy: 0.7434
Epoch 10/30
124/124 [==============================] - 1s 11ms/step - loss: 0.0224 - accuracy: 0.7009 - val_loss: 0.0187 - val_accuracy: 0.7542
Epoch 11/30
124/124 [==============================] - 1s 11ms/step - loss: 0.0216 - accuracy: 0.7277 - val_loss: 0.0180 - val_accuracy: 0.7693
Epoch 12/30
124/124 [==============================] - 1s 10ms/step - loss: 0.0209 - accuracy: 0.7547 - val_loss: 0.0178 - val_accuracy: 0.7

Epoch 2/30
124/124 [==============================] - 1s 10ms/step - loss: 0.0406 - accuracy: 0.5839 - val_loss: 0.0352 - val_accuracy: 0.3613
Epoch 3/30
124/124 [==============================] - 1s 10ms/step - loss: 0.0364 - accuracy: 0.5849 - val_loss: 0.0313 - val_accuracy: 0.3614
Epoch 4/30
124/124 [==============================] - 1s 10ms/step - loss: 0.0325 - accuracy: 0.6048 - val_loss: 0.0269 - val_accuracy: 0.3628
Epoch 5/30
124/124 [==============================] - 1s 10ms/step - loss: 0.0299 - accuracy: 0.6144 - val_loss: 0.0234 - val_accuracy: 0.6004
Epoch 6/30
124/124 [==============================] - 1s 10ms/step - loss: 0.0274 - accuracy: 0.6792 - val_loss: 0.0217 - val_accuracy: 0.6856
Epoch 7/30
124/124 [==============================] - 1s 10ms/step - loss: 0.0255 - accuracy: 0.6926 - val_loss: 0.0206 - val_accuracy: 0.6300
Epoch 8/30
124/124 [==============================] - 1s 10ms/step - loss: 0.0241 - accuracy: 0.6987 - val_loss: 0.0201 - val_accuracy: 0.6845

Epoch 7/30
124/124 [==============================] - 1s 11ms/step - loss: 0.0255 - accuracy: 0.6362 - val_loss: 0.0227 - val_accuracy: 0.6726
Epoch 8/30
124/124 [==============================] - 1s 10ms/step - loss: 0.0245 - accuracy: 0.6590 - val_loss: 0.0213 - val_accuracy: 0.6805
Epoch 9/30
124/124 [==============================] - 1s 11ms/step - loss: 0.0239 - accuracy: 0.6865 - val_loss: 0.0205 - val_accuracy: 0.6784
Epoch 10/30
124/124 [==============================] - 1s 10ms/step - loss: 0.0228 - accuracy: 0.6908 - val_loss: 0.0184 - val_accuracy: 0.6768
Epoch 11/30
124/124 [==============================] - 1s 11ms/step - loss: 0.0220 - accuracy: 0.7219 - val_loss: 0.0172 - val_accuracy: 0.7330
Epoch 12/30
124/124 [==============================] - 1s 11ms/step - loss: 0.0218 - accuracy: 0.7103 - val_loss: 0.0176 - val_accuracy: 0.7100
Epoch 13/30
124/124 [==============================] - 1s 11ms/step - loss: 0.0209 - accuracy: 0.7306 - val_loss: 0.0163 - val_accuracy: 0.

# 3. Train the LSTM classification model w/ AUGMENTED dataset included

In [9]:
DATA = get_filenames("ScooterData")

DATA_AUG = get_filenames("AugData")
DATA_COMB = DATA + DATA_AUG

OPTIONS = {
    "preprocess": StableFilter(stable_label=0, padding=30),
    "batchsize": 40,
    "timestamp": 16,
    "optimizer": "adam",
}

MAX_EPOCHS = 30

number_of_test_file = 4
stable_test_data = StableFilter(stable_label=0, padding=30).transform(split_data(DATA_COMB[:number_of_test_file], 0, 0)[0])
unstable_test_data = UnstableFilter(stable_label=0, padding=10).transform(split_data(DATA_COMB[:number_of_test_file], 0, 0)[0])

SETTINGS = {
    "max_epochs":MAX_EPOCHS,
    "valid_ratio":0.3,
    "test_ratio":0,
    "early_stop_valid_patience":MAX_EPOCHS//10,
    "early_stop_train_patience":MAX_EPOCHS//10,
    "num_train_per_config":10,
    "loss":'mae',
    "metrics": ['mae'],
    # "loss":"sparse_categorical_crossentropy",
    "metrics": ['accuracy'],
    "verbose": 1,
    "test_data": [unstable_test_data, stable_test_data]
}

ModelTrain(Encoder_Decoder, DATA_COMB[number_of_test_file:], OPTIONS, **SETTINGS).run()

batchsize   : 40
timestamp   : 16
optimizer   : adam
preprocess  : Stable Filter

(5484, 16, 27)
Model: "model_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, 16, 27)]          0         
                                                                 
 lstm_6 (LSTM)               (None, 10)                1520      
                                                                 
 repeat_vector_1 (RepeatVect  (None, 16, 10)           0         
 or)                                                             
                                                                 
 lstm_7 (LSTM)               (None, 16, 27)            4104      
                                                                 
Total params: 5,624
Trainable params: 5,624
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
138/138 [=====

In [10]:
MAX_EPOCHS = 30

OPTIONS = {
    "preprocess": [StableFilter(stable_label=0, padding=30)],
    "batchsize": [40],
    "timestamp": [16],
    "optimizer": ["adam"],
    "layer1": [{"units": i*5} for i in range(1, 10)],
}

number_of_test_file = 4
stable_test_data = StableFilter(stable_label=0, padding=30).transform(split_data(DATA_COMB[:number_of_test_file], 0, 0)[0])
unstable_test_data = UnstableFilter(stable_label=0, padding=10).transform(split_data(DATA_COMB[:number_of_test_file], 0, 0)[0])

SETTINGS = {
    "max_epochs":MAX_EPOCHS,
    "valid_ratio":0.3,
    "test_ratio":0,
    "early_stop_valid_patience":MAX_EPOCHS//10,
    "early_stop_train_patience":MAX_EPOCHS//10,
    "num_train_per_config":10,
    "loss":'mae',
    "metrics": ['mae'],
    # "loss":"sparse_categorical_crossentropy",
    "metrics": ['accuracy'],
    "verbose": 1,
    "test_data": [unstable_test_data, stable_test_data]
}

ModelTest(Encoder_Decoder, DATA_COMB[number_of_test_file:], OPTIONS, **SETTINGS).run()

preprocess  : Stable Filter
batchsize   : 40
timestamp   : 16
optimizer   : adam
layer1      : {'units': 5}

(5484, 16, 27)
Model: "model_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_18 (InputLayer)       [(None, 16, 27)]          0         
                                                                 
 lstm_6 (LSTM)               (None, 5)                 660       
                                                                 
 repeat_vector_1 (RepeatVect  (None, 16, 5)            0         
 or)                                                             
                                                                 
 lstm_7 (LSTM)               (None, 16, 27)            3564      
                                                                 
Total params: 4,224
Trainable params: 4,224
Non-trainable params: 0
________________________________________________________________

Epoch 5/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0407 - accuracy: 0.5440 - val_loss: 0.0315 - val_accuracy: 0.5897
Epoch 6/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0387 - accuracy: 0.5449 - val_loss: 0.0308 - val_accuracy: 0.6302
Epoch 7/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0371 - accuracy: 0.5478 - val_loss: 0.0301 - val_accuracy: 0.6233
Epoch 8/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0358 - accuracy: 0.5471 - val_loss: 0.0294 - val_accuracy: 0.6022
Epoch 9/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0349 - accuracy: 0.5449 - val_loss: 0.0288 - val_accuracy: 0.5910
Epoch 10/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0342 - accuracy: 0.5422 - val_loss: 0.0284 - val_accuracy: 0.5797
Epoch 11/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0337 - accuracy: 0.5388 - val_loss: 0.0282 - val_accuracy: 0.5676
Epoc

138/138 [==============================] - 1s 9ms/step - loss: 0.0503 - accuracy: 0.5145 - val_loss: 0.0377 - val_accuracy: 0.4927
Epoch 3/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0471 - accuracy: 0.4899 - val_loss: 0.0349 - val_accuracy: 0.4614
Epoch 4/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0446 - accuracy: 0.5009 - val_loss: 0.0328 - val_accuracy: 0.5230
Epoch 5/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0416 - accuracy: 0.4928 - val_loss: 0.0313 - val_accuracy: 0.5803
Epoch 6/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0392 - accuracy: 0.4936 - val_loss: 0.0299 - val_accuracy: 0.5976
Epoch 7/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0375 - accuracy: 0.4983 - val_loss: 0.0288 - val_accuracy: 0.5814
Epoch 8/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0361 - accuracy: 0.4933 - val_loss: 0.0276 - val_accuracy: 0.5218
Epoch 9/30
138/13

138/138 [==============================] - 1s 9ms/step - loss: 0.0262 - accuracy: 0.6953 - val_loss: 0.0200 - val_accuracy: 0.6870
Epoch 30/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0259 - accuracy: 0.6981 - val_loss: 0.0198 - val_accuracy: 0.6932
Epoch 1/30
138/138 [==============================] - 4s 14ms/step - loss: 0.0845 - accuracy: 0.3883 - val_loss: 0.0467 - val_accuracy: 0.4612
Epoch 2/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0522 - accuracy: 0.5018 - val_loss: 0.0391 - val_accuracy: 0.4064
Epoch 3/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0473 - accuracy: 0.4925 - val_loss: 0.0343 - val_accuracy: 0.3229
Epoch 4/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0427 - accuracy: 0.5045 - val_loss: 0.0313 - val_accuracy: 0.3955
Epoch 5/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0394 - accuracy: 0.5495 - val_loss: 0.0294 - val_accuracy: 0.4493
Epoch 6/30
138/

138/138 [==============================] - 1s 9ms/step - loss: 0.0291 - accuracy: 0.6435 - val_loss: 0.0235 - val_accuracy: 0.7162
Epoch 27/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0287 - accuracy: 0.6570 - val_loss: 0.0230 - val_accuracy: 0.7187
Epoch 28/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0283 - accuracy: 0.6660 - val_loss: 0.0225 - val_accuracy: 0.7214
Epoch 29/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0279 - accuracy: 0.6742 - val_loss: 0.0220 - val_accuracy: 0.7264
Epoch 30/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0275 - accuracy: 0.6779 - val_loss: 0.0217 - val_accuracy: 0.7290
Epoch 1/30
138/138 [==============================] - 4s 13ms/step - loss: 0.0831 - accuracy: 0.2869 - val_loss: 0.0452 - val_accuracy: 0.4421
Epoch 2/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0523 - accuracy: 0.4528 - val_loss: 0.0381 - val_accuracy: 0.3706
Epoch 3/30
1

...

138/138 [==============================] - 1s 9ms/step - loss: 0.0301 - accuracy: 0.6367 - val_loss: 0.0206 - val_accuracy: 0.6755
Epoch 5/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0280 - accuracy: 0.6566 - val_loss: 0.0190 - val_accuracy: 0.7344
Epoch 6/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0256 - accuracy: 0.7079 - val_loss: 0.0184 - val_accuracy: 0.7553
Epoch 7/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0241 - accuracy: 0.7241 - val_loss: 0.0179 - val_accuracy: 0.7398
Epoch 8/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0230 - accuracy: 0.7347 - val_loss: 0.0174 - val_accuracy: 0.7099
Epoch 9/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0220 - accuracy: 0.7422 - val_loss: 0.0160 - val_accuracy: 0.7315
Epoch 10/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0211 - accuracy: 0.7547 - val_loss: 0.0162 - val_accuracy: 0.7366
Epoch 11/30
138/

138/138 [==============================] - 4s 13ms/step - loss: 0.0530 - accuracy: 0.5254 - val_loss: 0.0315 - val_accuracy: 0.4294
Epoch 2/30
138/138 [==============================] - 1s 10ms/step - loss: 0.0372 - accuracy: 0.5552 - val_loss: 0.0260 - val_accuracy: 0.6169
Epoch 3/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0333 - accuracy: 0.5715 - val_loss: 0.0247 - val_accuracy: 0.6134
Epoch 4/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0312 - accuracy: 0.5652 - val_loss: 0.0236 - val_accuracy: 0.6351
Epoch 5/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0292 - accuracy: 0.5691 - val_loss: 0.0224 - val_accuracy: 0.6449
Epoch 6/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0270 - accuracy: 0.5926 - val_loss: 0.0202 - val_accuracy: 0.6813
Epoch 7/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0254 - accuracy: 0.6327 - val_loss: 0.0190 - val_accuracy: 0.6961
Epoch 8/30
138/

138/138 [==============================] - 1s 9ms/step - loss: 0.0165 - accuracy: 0.8245 - val_loss: 0.0114 - val_accuracy: 0.8347
Epoch 29/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0163 - accuracy: 0.8232 - val_loss: 0.0113 - val_accuracy: 0.8330
Epoch 30/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0162 - accuracy: 0.8247 - val_loss: 0.0112 - val_accuracy: 0.8410
Epoch 1/30
138/138 [==============================] - 4s 14ms/step - loss: 0.0568 - accuracy: 0.5848 - val_loss: 0.0327 - val_accuracy: 0.5946
Epoch 2/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0390 - accuracy: 0.6150 - val_loss: 0.0303 - val_accuracy: 0.5813
Epoch 3/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0353 - accuracy: 0.6066 - val_loss: 0.0253 - val_accuracy: 0.6018
Epoch 4/30
138/138 [==============================] - 1s 10ms/step - loss: 0.0318 - accuracy: 0.6106 - val_loss: 0.0222 - val_accuracy: 0.6482
Epoch 5/30
13

138/138 [==============================] - 1s 10ms/step - loss: 0.0166 - accuracy: 0.8046 - val_loss: 0.0116 - val_accuracy: 0.8179
Epoch 26/30
138/138 [==============================] - 1s 10ms/step - loss: 0.0165 - accuracy: 0.8082 - val_loss: 0.0114 - val_accuracy: 0.8207
Epoch 27/30
138/138 [==============================] - 1s 10ms/step - loss: 0.0167 - accuracy: 0.8104 - val_loss: 0.0114 - val_accuracy: 0.8205
Epoch 28/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0163 - accuracy: 0.8082 - val_loss: 0.0113 - val_accuracy: 0.8187
Epoch 29/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0163 - accuracy: 0.8110 - val_loss: 0.0113 - val_accuracy: 0.8248
Epoch 30/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0164 - accuracy: 0.8113 - val_loss: 0.0111 - val_accuracy: 0.8200
Epoch 1/30
138/138 [==============================] - 4s 14ms/step - loss: 0.0559 - accuracy: 0.4757 - val_loss: 0.0333 - val_accuracy: 0.4337
Epoch 2/

138/138 [==============================] - 1s 9ms/step - loss: 0.0170 - accuracy: 0.8018 - val_loss: 0.0119 - val_accuracy: 0.8434
Epoch 24/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0170 - accuracy: 0.7914 - val_loss: 0.0119 - val_accuracy: 0.8406
Epoch 25/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0166 - accuracy: 0.8061 - val_loss: 0.0117 - val_accuracy: 0.8261
Epoch 26/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0165 - accuracy: 0.8047 - val_loss: 0.0115 - val_accuracy: 0.8199
Epoch 27/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0165 - accuracy: 0.8032 - val_loss: 0.0119 - val_accuracy: 0.8248
Epoch 28/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0165 - accuracy: 0.8016 - val_loss: 0.0116 - val_accuracy: 0.8375
Epoch 29/30
138/138 [==============================] - 1s 9ms/step - loss: 0.0164 - accuracy: 0.8056 - val_loss: 0.0116 - val_accuracy: 0.8382
Epoch 29: e